In [23]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [24]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
# pd.options.display.float_format = '{:.2f}'.format

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
link1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_2.7.csv'
link2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_2.7.csv'
normal = pd.read_csv(link1)
covid = pd.read_csv(link2)

In [27]:
normal.columns, covid.columns, normal.shape, covid.shape

(Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
        'latitude', 'longitude', 'community_board', 'council_district',
        'census_tract', 'bin', 'nta', 'year', 'month_year', 'geometry',
        'yearbuilt', 'bldgclass', 'numfloors', 'unitsres', 'ownername',
        'bldgarea', 'building_type', 'building_category', 'is_condo',
        'floor_category', 'rent_era', 'architectural_style', 'economic_period',
        'residential_units_category', 'is_llc', 'building_size_category',
        'size_quartile', 'decade', 'fips', 'e_totpop', 'rpl_theme1',
        'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150',
        'ep_unemp', 'ep_nohsdp', 'ep_uninsur', 'ep_age65', 'ep_age17',
        'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam',
        'ep_hisp', 'ep_asian', 'ep_aian', 'ep_n

# **Add buildings_affected column**

In [28]:
evictions_per_nta = normal.groupby('nta')['bin'].nunique().reset_index(name='buildings_affected')
evictions_per_nta.head()

,nta,buildings_affected
0,Allerton-Pelham Gardens,97
1,Annadale-Huguenot-Prince's Bay-Eltingville,15
2,Arden Heights,16
3,Astoria,267
4,Auburndale,24


In [29]:
unique_buildings = normal.drop_duplicates(subset=['nta', 'bin'])
unique_buildings.head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_unit_count,average_year_eviction_count,average_year_eviction_nta_count,evictions_nta_per_1k,evictions_nta_per_unit_per_1k
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,0.021692
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,0.300000,0.6,38.0,0.408567,0.118460
2,*324973/22_5308,3057820030,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,0.150000,0.6,39.6,0.425770,0.160426
3,*53336/16_170279,2032510420,*53336/16,170279,3400 PAUL AVENUE,15D,2018-10-17,BRONX,10468,Not an Ejectment,Possession,40.877190,-73.889569,7.0,11.0,409.0,2015444,Van Cortlandt Village,2018,2018-10,POINT (-73.889569 40.87719),1967.0,D4,21.0,352.0,SCOTT TOWER HOUSING CO INC,381213.0,post-war,condo-co-op,True,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,10468,81397.0,0.9954,0.9407,0.9870

In [30]:
total_units_per_nta = unique_buildings.groupby('nta')['unitsres'].sum().reset_index(name='total_units_per_nta')
total_units_per_nta.head()

,nta,total_units_per_nta
0,Allerton-Pelham Gardens,8248.0
1,Annadale-Huguenot-Prince's Bay-Eltingville,31.0
2,Arden Heights,27.0
3,Astoria,20683.0
4,Auburndale,225.0


In [31]:
building_units_per_nta = pd.merge(evictions_per_nta, total_units_per_nta, on='nta', how='left')
building_units_per_nta.head()

,nta,buildings_affected,total_units_per_nta
0,Allerton-Pelham Gardens,97,8248.0
1,Annadale-Huguenot-Prince's Bay-Eltingville,15,31.0
2,Arden Heights,16,27.0
3,Astoria,267,20683.0
4,Auburndale,24,225.0


In [32]:
normal = normal.merge(building_units_per_nta, on='nta', how='left')
normal.shape

(66397, 98)

In [33]:
normal.columns

Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'nta', 'year', 'month_year', 'geometry',
       'yearbuilt', 'bldgclass', 'numfloors', 'unitsres', 'ownername',
       'bldgarea', 'building_type', 'building_category', 'is_condo',
       'floor_category', 'rent_era', 'architectural_style', 'economic_period',
       'residential_units_category', 'is_llc', 'building_size_category',
       'size_quartile', 'decade', 'fips', 'e_totpop', 'rpl_theme1',
       'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150',
       'ep_unemp', 'ep_nohsdp', 'ep_uninsur', 'ep_age65', 'ep_age17',
       'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam',
       'ep_hisp', 'ep_asian', 'ep_aian', 'ep_nhpi', 'ep_twom

In [34]:
# normal['evictions_nta_per_1k'] = normal['evictions_nta_per_1k'] /5

In [35]:
normal[['nta','average_year_eviction_count', 'average_year_eviction_unit_count']].head(10)
# for the same nta, they likely have different values for these two columns, because they were groupedby bin

,nta,average_year_eviction_count,average_year_eviction_unit_count
0,East New York,0.8,0.266667
1,Sunset Park East,0.6,0.300000
2,Sunset Park West,0.6,0.150000
3,Van Cortlandt Village,0.8,0.002273
4,Mott Haven-Port Morris,1.6,0.024615
5,Bedford Park-Fordham North,1.6,0.038095
6,Bedford Park-Fordham North,1.0,0.500000
7,Claremont-Bathgate,1.6,0.048485
8,East New York,0.4,0.080000
9,Brighton Beach,1.2,0.007692


In [36]:
normal[['nta', 'average_year_eviction_nta_count', 'evictions_nta_per_1k','evictions_nta_per_unit_per_1k']].head(10)
# this is correct

,nta,average_year_eviction_nta_count,evictions_nta_per_1k,evictions_nta_per_unit_per_1k
0,East New York,266.0,2.747905,0.021692
1,Sunset Park East,38.0,0.408567,0.118460
2,Sunset Park West,39.6,0.425770,0.160426
3,Van Cortlandt Village,172.8,2.122928,0.028336
4,Mott Haven-Port Morris,158.8,3.578188,0.031756
5,Bedford Park-Fordham North,306.0,3.759352,0.052102
6,Bedford Park-Fordham North,306.0,3.701105,0.052102
7,Claremont-Bathgate,125.6,1.573600,0.022545
8,East New York,266.0,2.458865,0.021692
9,Brighton Beach,44.0,0.529680,0.035449


sum all evictions per NTA -> divide by years -> average_year_eviction_nta_count. \
sum all units per NTA (deduplicate buildings to avoid overcounting). \
divide average_year_eviction_nta_count by total units -> evictions per unit per year. \
divide by NTA population and multiply by 1,000 -> evictions per unit per 1k residents. \

In [37]:
file_path3 = '/content/drive/My Drive/X999/bbl_cleaned.csv'
bbl_cleaned = pd.read_csv(file_path3)

In [38]:
bbl_cleaned.columns
# there is no nta in this dataset either, so it would be error-prone to approximate the nta data

Index(['borough', 'block', 'lot', 'community board', 'census tract 2010',
       'cb2010', 'schooldist', 'council district', 'postcode', 'firecomp',
       'policeprct', 'healtharea', 'sanitboro', 'sanitsub', 'address',
       'zonedist1', 'zonedist2', 'zonedist3', 'overlay1', 'overlay2',
       'spdist1', 'ltdheight', 'splitzone', 'bldgclass', 'landuse',
       'easements', 'ownertype', 'ownername', 'lotarea', 'bldgarea', 'comarea',
       'resarea', 'officearea', 'retailarea', 'garagearea', 'strgearea',
       'factryarea', 'otherarea', 'areasource', 'numbldgs', 'numfloors',
       'unitsres', 'unitstotal', 'lotfront', 'lotdepth', 'bldgfront',
       'bldgdepth', 'ext', 'proxcode', 'irrlotcode', 'lottype', 'bsmtcode',
       'assessland', 'assesstot', 'exempttot', 'yearbuilt', 'yearalter1',
       'yearalter2', 'histdist', 'landmark', 'builtfar', 'residfar', 'commfar',
       'facilfar', 'borocode', 'bbl', 'condono', 'tract2010', 'xcoord',
       'ycoord', 'latitude', 'longitude', 'z

In [40]:
# the new way:
# stop
# normal['evictions_nta_per_unit_per_1k'] = (normal['average_year_eviction_nta_count'] / (normal['total_units_per_nta'] * normal['e_totpop'])) * 1000

In [42]:
normal[['nta', 'evictions_nta_per_unit_per_1k', 'average_year_eviction_nta_count', 'total_units_per_nta']].head(10)

# but this is still an inflated number,
# because for the buildings never appeared in the eviction dataset, their buildings and units were not counted at all
# we would have to have this limitation because bbl dataset did not have nta, and if we were to correctly include all buildings in
# each nta, we will need to derive the nta from community_board and census_tract, which will more likely to introduce errors and misjudgements
# than using only the buildings affected and were actually in the eviction dataset.

,nta,evictions_nta_per_unit_per_1k,average_year_eviction_nta_count,total_units_per_nta
0,East New York,0.021692,266.0,50344.0
1,Sunset Park East,0.118460,38.0,2144.0
2,Sunset Park West,0.160426,39.6,1601.0
3,Van Cortlandt Village,0.028336,172.8,15614.0
4,Mott Haven-Port Morris,0.031756,158.8,38635.0
5,Bedford Park-Fordham North,0.052102,306.0,16180.0
6,Bedford Park-Fordham North,0.052102,306.0,16180.0
7,Claremont-Bathgate,0.022545,125.6,16920.0
8,East New York,0.021692,266.0,50344.0
9,Brighton Beach,0.035449,44.0,6662.0


# **SVI measure analysis**

four types of aggregated and groupedby analysis:

average eviction count per building, per year;

average eviction count per nta population;

average eviction count per unit per building per year;

average eviction count per unit per nta population.

SVI measures:

ep_age17: age 17 or younger.
possible social causes impacted: homelessness, change of schools, education impact; \

ep_age65: age 65 and above.
possible social causes impacted: homelessness;\

ep_unemp: unemployed pct.
possible social causes impacted: homelessness;\





In [43]:
analysis_columns = normal[['primary_key', 'bin', 'bbl', 'latitude', 'longitude', 'eviction_address', 'zipcode', 'borough', 'nta','average_year_eviction_count', 	'average_year_eviction_unit_count',
                              'average_year_eviction_nta_count',	'evictions_nta_per_1k', 'evictions_nta_per_unit_per_1k',
                              'unitsres', 'e_totpop']]
analysis_columns.head(1)

,primary_key,bin,bbl,latitude,longitude,eviction_address,zipcode,borough,nta,average_year_eviction_count,average_year_eviction_unit_count,average_year_eviction_nta_count,evictions_nta_per_1k,evictions_nta_per_unit_per_1k,unitsres,e_totpop
0,*308072/22_5865,3083989,3037420029,40.672121,-73.891105,356 MILLER AVE,11207,BROOKLYN,East New York,0.8,0.266667,266.0,2.747905,0.021692,3.0,96801.0


In [44]:
normal['ep_age17'].head(1)
# pct

,ep_age17
0,22.5


# **Groupby and aggregate first**

In [45]:
svi_analysis_columns = ['ep_age17', 'ep_age65', 'ep_unemp', 'ep_afam', 'ep_hisp', 'evictions_nta_per_1k', 'evictions_nta_per_unit_per_1k',
                        'average_year_eviction_count', 'average_year_eviction_unit_count', 'average_year_eviction_nta_count']

## **Children**

In [46]:
normal[['nta', 'primary_key', 'eviction_address', 'average_year_eviction_nta_count']].sort_values('nta', ascending=True).head(10)

,nta,primary_key,eviction_address,average_year_eviction_nta_count
58618,Allerton-Pelham Gardens,B802448/17_389388,1140 BURKE AVENUE,25.4
58619,Allerton-Pelham Gardens,B802450/17_397880,1240 BURKE AVENUE,25.4
16884,Allerton-Pelham Gardens,312913/21_362926,2325 BOUCK AVE,25.4
23563,Allerton-Pelham Gardens,330196/22_363899,1256 EAST GUN HILL ROAD,25.4
7991,Allerton-Pelham Gardens,29536/19_98103,2958 RADCLIFF AVENUE,25.4
8003,Allerton-Pelham Gardens,29596/19_356492,1114 ADEE AVENUE,25.4
8043,Allerton-Pelham Gardens,29730/19_356158,2219 MORGAN AVENUE,25.4
8084,Allerton-Pelham Gardens,29877/17_337087,2934 WILSON AVENUE,25.4
57080,Allerton-Pelham Gardens,B38861/18_108676,3018 PAULDING AVENUE,25.4
27763,Allerton-Pelham Gardens,47208/18_20541,1135 PELHAM PARKWAY NORTH,25.4


In [47]:
normal.groupby('nta')[svi_analysis_columns].mean().head(10)
# using mean() here, because the average_year_eviction_nta_count might be slightly different due to zipcode
# svi measures. Here, we just use mean to offset some tiny deviations and get the most accurate one as much as possible

,ep_age17,ep_age65,ep_unemp,ep_afam,ep_hisp,evictions_nta_per_1k,evictions_nta_per_unit_per_1k,average_year_eviction_count,average_year_eviction_unit_count,average_year_eviction_nta_count
nta,,,,,,,,,,
Allerton-Pelham Gardens,22.211024,16.955906,8.409449,51.682677,29.820472,0.354234,0.031814,0.316535,0.126290,25.4
Annadale-Huguenot-Prince's Bay-Eltingville,22.429412,17.382353,4.041176,0.676471,11.482353,0.071809,2.965754,0.247059,0.195294,3.4
Arden Heights,22.100000,18.000000,4.000000,0.800000,12.800000,0.053566,1.847106,0.223529,0.141176,3.4
Astoria,13.776205,12.541265,7.907530,4.771687,26.680422,1.800873,0.058503,0.296386,0.031679,66.4
Auburndale,19.805556,18.638889,6.844444,4.783333,18.633333,0.177189,0.283015,0.411111,0.117100,7.2
Baisley Park,21.366667,14.332129,8.388755,68.014056,12.676305,1.525162,0.106163,0.368675,0.196634,49.8
Bath Beach,21.482635,18.013174,7.144910,1.568263,17.340719,0.384045,0.009234,0.564072,0.023973,33.4
Battery Park City-Lower Manhattan,12.024603,8.515079,3.301587,5.113492,9.581746,2.959166,0.032727,1.561905,0.003770,25.2
Bay Ridge,20.368932,17.892961,5.900971,2.942961,21.999515,1.118261,0.043087,0.484466,0.035094,82.4


In [48]:
# rate of children affected per 1000 people in the population in each nta
normal['children_impacted_nta_per_1k'] = (normal['evictions_nta_per_1k'] * (normal['ep_age17'] / 100))
# normal['children_affected_per_1k_2'] = ((normal['average_year_eviction_nta_count'] /normal['e_totpop'])*1000 * (normal['ep_age17'] / 100))
normal[['nta', 'children_impacted_nta_per_1k']].head(10)

,nta,children_impacted_nta_per_1k
0,East New York,0.618279
1,Sunset Park East,0.103776
2,Sunset Park West,0.108146
3,Van Cortlandt Village,0.560453
4,Mott Haven-Port Morris,1.005471
5,Bedford Park-Fordham North,0.992469
6,Bedford Park-Fordham North,0.951184
7,Claremont-Bathgate,0.464212
8,East New York,0.651599
9,Brighton Beach,0.090575


## **Unemployment**

In [49]:
normal['unemployed_impacted_nta_per_1k'] = normal['evictions_nta_per_1k'] * normal['ep_unemp']/100
normal['unemployed_impacted_nta_per_1k'].head()

,unemployed_impacted_nta_per_1k
0,0.305018
1,0.030643
2,0.031933
3,0.246260
4,0.447274


## **Elderly**

In [50]:
normal['elderly_impacted_nta_per_1k'] = normal['evictions_nta_per_1k'] * normal['ep_age65']/100
normal['elderly_impacted_nta_per_1k'].head()

,elderly_impacted_nta_per_1k
0,0.379211
1,0.053522
2,0.055776
3,0.237768
4,0.361397


## **black and hispanics impacted**

In [51]:
normal['bh_impacted_nta_per_1k'] = normal['evictions_nta_per_1k'] * (normal['ep_afam'] + normal['ep_hisp'])/100
normal['bh_impacted_nta_per_1k'].head()

,bh_impacted_nta_per_1k
0,2.437392
1,0.174050
2,0.181378
3,1.987061
4,3.406435


# **housing burden areas impacted**

defintion: Housing cost-burdened occupied housing
units with annual income less than $75,000
(30%+ of income spent on housing costs)
estimate, 2018-2022 ACS
source: https://www.atsdr.cdc.gov/place-health/media/pdfs/2024/10/SVI2022Documentation.pdf

It is the pct of households that spend more tha 30% of income on housing costs.

In [59]:
normal[['nta', 'ep_hburd']].sort_values('ep_hburd', ascending=True).head()

,nta,ep_hburd
29883,East Elmhurst,-999.0
65395,St. Albans,-999.0
11328,Battery Park City-Lower Manhattan,6.4
49506,Battery Park City-Lower Manhattan,6.4
18842,Battery Park City-Lower Manhattan,6.4


In [61]:
Albans = normal[normal['nta'] == 'St. Albans']

In [63]:
normal.loc[normal['nta'] == 'St. Albans', 'ep_hburd'] =  64.2
# https://anhd.org/report/how-affordable-housing-threatened-your-neighborhood-2020/
# https://www.nyc.gov/assets/doh/downloads/pdf/data/2018chp-qn12.pdf
normal.loc[normal['nta'] == 'East Elmhurst', 'ep_hburd'] = 54.2

In [84]:
normal['hburd_impacted_nta_per_1k'] = normal['evictions_nta_per_1k'] * normal['ep_hburd']/100
normal['hburd_impacted_nta_per_1k'].head()

,hburd_impacted_nta_per_1k
0,1.228314
1,0.178135
2,0.185636
3,1.203700
4,1.857080


# **Analysis**

## **Children Analysis: The most vulnerable nta for kids (high likelihood for change of schools or dropping out)**

In [87]:
svi_analysis_columns = ['children_impacted_nta_per_1k', 'elderly_impacted_nta_per_1k', 'unemployed_impacted_nta_per_1k', 'bh_impacted_nta_per_1k', 'hburd_impacted_nta_per_1k']

In [88]:
svi_analysis_df = normal.groupby('nta')[svi_analysis_columns].mean()
svi_analysis_df.head()

,children_impacted_nta_per_1k,elderly_impacted_nta_per_1k,unemployed_impacted_nta_per_1k,bh_impacted_nta_per_1k,hburd_impacted_nta_per_1k
nta,,,,,
Allerton-Pelham Gardens,0.078668,0.060086,0.029748,0.288334,0.129424
Annadale-Huguenot-Prince's Bay-Eltingville,0.016192,0.012321,0.002913,0.008356,0.020183
Arden Heights,0.011838,0.009642,0.002143,0.007285,0.012320
Astoria,0.245097,0.223135,0.141748,0.560924,0.589607
Auburndale,0.034827,0.033327,0.012162,0.040229,0.059799


In [67]:
children_top_15 = svi_analysis_df.sort_values('children_impacted_nta_per_1k', ascending=False)['children_impacted_nta_per_1k'].head(15)
children_bottom_15 = svi_analysis_df.sort_values('children_impacted_nta_per_1k', ascending=True)['children_impacted_nta_per_1k'].head(15)
children_top_15

,children_impacted_nta_per_1k
nta,
Central Harlem North-Polo Grounds,2.222442
Hunts Point,1.747318
East Concourse-Concourse Village,1.002249
Mott Haven-Port Morris,0.995718
Crown Heights North,0.964865
Bedford Park-Fordham North,0.959010
Mount Hope,0.928392
East Tremont,0.899919
University Heights-Morris Heights,0.892011


In [68]:
children_bottom_15

,children_impacted_nta_per_1k
nta,
park-cemetery-etc-Brooklyn,0.001224
Arden Heights,0.011838
Brooklyn Heights-Cobble Hill,0.012111
Annadale-Huguenot-Prince's Bay-Eltingville,0.016192
park-cemetery-etc-Bronx,0.018648
Queensboro Hill,0.018719
Rossville-Woodrow,0.021168
Windsor Terrace,0.021527
Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,0.022684


In [69]:
children_top_15 = children_top_15.to_frame()
children_bottom_15 = children_bottom_15.to_frame()
# series can't be transposed

In [70]:
children_top_15.T

nta,Central Harlem North-Polo Grounds,Hunts Point,East Concourse-Concourse Village,Mott Haven-Port Morris,Crown Heights North,Bedford Park-Fordham North,Mount Hope,East Tremont,University Heights-Morris Heights,East Harlem North,Woodlawn-Wakefield,Morrisania-Melrose,Williamsbridge-Olinville,Melrose South-Mott Haven North,West Concourse
children_impacted_nta_per_1k,2.222442,1.747318,1.002249,0.995718,0.964865,0.95901,0.928392,0.899919,0.892011,0.812748,0.786279,0.785742,0.768984,0.712541,0.678763


In [71]:
children_bottom_15.T

nta,park-cemetery-etc-Brooklyn,Arden Heights,Brooklyn Heights-Cobble Hill,Annadale-Huguenot-Prince's Bay-Eltingville,park-cemetery-etc-Bronx,Queensboro Hill,Rossville-Woodrow,Windsor Terrace,Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,Stuyvesant Town-Cooper Village,New Springville-Bloomfield-Travis,New Dorp-Midland Beach,Elmhurst-Maspeth,East Flushing,Glen Oaks-Floral Park-New Hyde Park
children_impacted_nta_per_1k,0.001224,0.011838,0.012111,0.016192,0.018648,0.018719,0.021168,0.021527,0.022684,0.028404,0.032056,0.032262,0.032414,0.033455,0.034189


## **Elderly analysis: The most vulnerable nta for elderly (more demand for social security/welfare)**

In [72]:
elderly_top_15 = svi_analysis_df.sort_values('elderly_impacted_nta_per_1k', ascending=False)['elderly_impacted_nta_per_1k'].head(15)
elderly_bottom_15 = svi_analysis_df.sort_values('elderly_impacted_nta_per_1k', ascending=True)['elderly_impacted_nta_per_1k'].head(15)
elderly_top_15 = elderly_top_15.to_frame()
elderly_bottom_15 = elderly_bottom_15.to_frame()

In [73]:
elderly_top_15.T

nta,Central Harlem North-Polo Grounds,Woodlawn-Wakefield,Starrett City,East Harlem North,Lenox Hill-Roosevelt Island,Hunts Point,Crown Heights North,Washington Heights North,Washington Heights South,Williamsbridge-Olinville,Seagate-Coney Island,Clinton,Marble Hill-Inwood,East Concourse-Concourse Village,Turtle Bay-East Midtown
elderly_impacted_nta_per_1k,1.570459,0.704522,0.668372,0.647124,0.632936,0.61781,0.572509,0.558458,0.545875,0.469725,0.466001,0.457286,0.45604,0.428843,0.423521


In [74]:
elderly_bottom_15.T

nta,park-cemetery-etc-Brooklyn,Brooklyn Heights-Cobble Hill,Arden Heights,park-cemetery-etc-Bronx,Windsor Terrace,Annadale-Huguenot-Prince's Bay-Eltingville,Rossville-Woodrow,Williamsburg,Queensboro Hill,Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,Westerleigh,Ocean Parkway South,Elmhurst-Maspeth,Glendale,Prospect Heights
elderly_impacted_nta_per_1k,0.000346,0.008602,0.009642,0.009719,0.011088,0.012321,0.015383,0.015511,0.018534,0.019316,0.022525,0.024256,0.024285,0.025979,0.02708


## **Unemployed analysis: The most vulnerable nta for elderly (more demand for social security/welfare, homelessness)**

In [75]:
unemployed_top_15 = svi_analysis_df.sort_values('unemployed_impacted_nta_per_1k', ascending=False)['unemployed_impacted_nta_per_1k'].head(15)
unemployed_bottom_15 = svi_analysis_df.sort_values('unemployed_impacted_nta_per_1k', ascending=True)['unemployed_impacted_nta_per_1k'].head(15)
unemployed_top_15 = unemployed_top_15.to_frame()
unemployed_bottom_15 = unemployed_bottom_15.to_frame()

In [76]:
unemployed_top_15.T

nta,Central Harlem North-Polo Grounds,Hunts Point,Bedford Park-Fordham North,East Concourse-Concourse Village,Mott Haven-Port Morris,East Tremont,Washington Heights North,University Heights-Morris Heights,Washington Heights South,Mount Hope,East Harlem North,Crown Heights North,Morrisania-Melrose,Williamsbridge-Olinville,Brownsville
unemployed_impacted_nta_per_1k,1.364225,0.7737,0.511399,0.491548,0.486258,0.47038,0.461236,0.443458,0.435368,0.432916,0.414562,0.411721,0.398061,0.339059,0.323669


In [77]:
unemployed_bottom_15.T

nta,park-cemetery-etc-Brooklyn,Arden Heights,Annadale-Huguenot-Prince's Bay-Eltingville,Brooklyn Heights-Cobble Hill,Rossville-Woodrow,Charleston-Richmond Valley-Tottenville,Windsor Terrace,Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,Queensboro Hill,New Springville-Bloomfield-Travis,East Elmhurst,park-cemetery-etc-Bronx,Oakland Gardens,Elmhurst-Maspeth,Ocean Parkway South
unemployed_impacted_nta_per_1k,0.000346,0.002143,0.002913,0.003716,0.003793,0.004904,0.0055,0.005867,0.006297,0.007374,0.008366,0.008692,0.008971,0.009195,0.009336


## **Black + Hispanic analysis: The most vulnerable nta for black and hispanic population**

In [78]:
bh_top_15 = svi_analysis_df.sort_values('bh_impacted_nta_per_1k', ascending=False)['bh_impacted_nta_per_1k'].head(15)
bh_bottom_15 = svi_analysis_df.sort_values('bh_impacted_nta_per_1k', ascending=True)['bh_impacted_nta_per_1k'].head(15)
bh_top_15 = bh_top_15.to_frame()
bh_bottom_15 = bh_bottom_15.to_frame()

In [79]:
bh_top_15.T

nta,Central Harlem North-Polo Grounds,Hunts Point,East Concourse-Concourse Village,Mott Haven-Port Morris,Bedford Park-Fordham North,East Harlem North,Crown Heights North,Mount Hope,University Heights-Morris Heights,East Tremont,Williamsbridge-Olinville,Morrisania-Melrose,Washington Heights South,Woodlawn-Wakefield,Melrose South-Mott Haven North
bh_impacted_nta_per_1k,9.438089,5.879795,3.536959,3.524135,3.288613,3.248847,3.114685,3.104834,3.099504,3.066189,2.833167,2.827551,2.650564,2.616902,2.510876


In [80]:
bh_bottom_15.T

nta,park-cemetery-etc-Brooklyn,Arden Heights,Annadale-Huguenot-Prince's Bay-Eltingville,Rossville-Woodrow,Brooklyn Heights-Cobble Hill,Windsor Terrace,Queensboro Hill,Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,Great Kills,Whitestone,New Springville-Bloomfield-Travis,New Dorp-Midland Beach,Ocean Parkway South,East Flushing,Charleston-Richmond Valley-Tottenville
bh_impacted_nta_per_1k,0.001399,0.007285,0.008356,0.009579,0.015414,0.020199,0.024646,0.02519,0.027571,0.027755,0.027842,0.03105,0.032365,0.034295,0.035214


# **Housing Burden areas impacted by evictions**

In [90]:
housing_burden_top_15 = svi_analysis_df.sort_values('hburd_impacted_nta_per_1k', ascending=False)['hburd_impacted_nta_per_1k'].head(15)
housing_burden_bottom_15 = svi_analysis_df.sort_values('hburd_impacted_nta_per_1k', ascending=True)['hburd_impacted_nta_per_1k'].head(15)
housing_burden_top_15 = housing_burden_top_15.to_frame()
housing_burden_bottom_15 = housing_burden_bottom_15.to_frame()

In [91]:
housing_burden_top_15.T

nta,Central Harlem North-Polo Grounds,Hunts Point,Bedford Park-Fordham North,Mott Haven-Port Morris,East Concourse-Concourse Village,East Harlem North,Crown Heights North,University Heights-Morris Heights,East Tremont,Mount Hope,Morrisania-Melrose,Woodlawn-Wakefield,Williamsbridge-Olinville,West Concourse,Washington Heights South
hburd_impacted_nta_per_1k,4.967754,3.394888,2.182264,2.072823,2.06362,1.863327,1.851733,1.827415,1.798963,1.778984,1.614065,1.562127,1.464709,1.444703,1.434616


In [92]:
housing_burden_bottom_15.T

nta,park-cemetery-etc-Brooklyn,Arden Heights,Brooklyn Heights-Cobble Hill,Annadale-Huguenot-Prince's Bay-Eltingville,Windsor Terrace,Rossville-Woodrow,Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill,Queensboro Hill,New Springville-Bloomfield-Travis,park-cemetery-etc-Bronx,New Dorp-Midland Beach,Great Kills,Oakland Gardens,Prospect Heights,Whitestone
hburd_impacted_nta_per_1k,0.001514,0.01232,0.012799,0.020183,0.025639,0.02917,0.02957,0.038703,0.040032,0.040773,0.041969,0.047165,0.048641,0.049891,0.05187


In [93]:
# change some of the analysis columns' names so they are more obvious
# building based: 'average_year_eviction_count'
# 'average_year_eviction_unit_count',
# 'average_year_eviction_nta_count',
# 'evictions_nta_per_1k',
# 'evictions_nta_per_unit_per_1k',
normal.rename(columns={'average_year_eviction_count':'average_year_eviction_count(building)',
                       'average_year_eviction_unit_count':'average_year_eviction_count_per_unit(building)',
                      'average_year_eviction_nta_count':'average_year_eviction_count_per_nta(nta)',
                      'evictions_nta_per_1k':'evictions_per_nta_1k(nta)',
                      'evictions_nta_per_unit_per_1k':'evictions_per_unit_nta_1k(nta)'}, inplace=True)

In [94]:
normal.head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1k(nta),evictions_per_unit_nta_1k(nta),buildings_affected,total_units_per_nta,children_impacted_nta_per_1k,unemployed_impacted_nta_per_1k,elderly_impacted_nta_per_1k,bh_impacted_nta_per_1k,hburd_nta_per_1k,hburd_impacted_nta_per_1k
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,0.021692,725,50344.0,0.618279,0.305018,0.379211,2.437392,1.228314,1.228314
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,0.300000,0.6,38.0,0.408567,0.118460,136,2144.0,0.103776,0.030643,0.053522,0.174050,0.178135,0.178135
2,*324973/22_5308,3057820030,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,0.150000,0.6,39.6,0.425770,0.160426,147,1601.0,0.108146,0.031933,0.055776,0.181378,0.185636,0.185636
3,*53336/16_170279,2032510420,*53336/16,170279,3400 PAUL AVENUE,15D,2018-10-17,BRONX,10468,Not 

In [102]:
# normal['average_year_eviction_per_building_nta(nta)'] = normal['average_year_eviction_count_per_nta(nta)'] / normal['buildings_affected'] * 100
# normal['average_year_eviction_per_building_nta(nta)'].head()
# every 100 buildings in an nta, what are their average eviction counts

## **Interpretations**
- Measures eviction intensity per building.
- Higher values indicate:
Fewer buildings are responsible for more evictions (concentration), or
Repeated evictions in the same buildings (e.g., due to high tenant turnover).


In [103]:
normal[['average_year_eviction_count(building)', 'average_year_eviction_count_per_unit(building)', 'average_year_eviction_count_per_nta(nta)',
        # 'average_year_eviction_per_building_nta(nta)',\
        'evictions_per_nta_1k(nta)', 'evictions_per_unit_nta_1k(nta)']]

,average_year_eviction_count(building),average_year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1k(nta),evictions_per_unit_nta_1k(nta)
0,0.8,0.266667,266.0,2.747905,0.021692
1,0.6,0.300000,38.0,0.408567,0.118460
2,0.6,0.150000,39.6,0.425770,0.160426
3,0.8,0.002273,172.8,2.122928,0.028336
4,1.6,0.024615,158.8,3.578188,0.031756
...,...,...,...,...,...
66392,3.8,0.008370,89.6,2.221616,0.044957
66393,0.4,0.200000,6.8,0.366241,0.133213
66394,0.4,0.200000,6.8,0.366241,0.133213
66395,0.2,0.000833,89.6,3.414764,0.044957


In [111]:
normal.columns

Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'ejectment',
       ...
       'evictions_per_nta_1k(nta)', 'evictions_per_unit_nta_1k(nta)',
       'buildings_affected', 'total_units_per_nta',
       'children_impacted_nta_per_1k', 'unemployed_impacted_nta_per_1k',
       'elderly_impacted_nta_per_1k', 'bh_impacted_nta_per_1k',
       'hburd_nta_per_1k', 'hburd_impacted_nta_per_1k'],
      dtype='object', length=104)

In [108]:
normal.drop(columns=['average_year_eviction_per_building_nta(nta)'], inplace=True)

In [116]:
# save the updated normal times data
normal.to_csv('/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_svi.csv', index=False)

In [115]:
normal.drop(columns=['hburd_nta_per_1k'], inplace=True)

In [117]:
normal.shape
# 103 is correct

(66397, 103)